In [ ]:
import requests
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()
USERNAME = os.getenv('Username')
PASSWORD = os.getenv('Password')

BASE_URL = 'https://www.space-track.org'
LOGIN_URL = f'{BASE_URL}/ajaxauth/login'

API_ENDPOINTS = {
    'latest_orbits': f'{BASE_URL}/basicspacedata/query/class/gp/EPOCH/>now-30/orderby/NORAD_CAT_ID/format/json',
    'recent_history_iss': f'{BASE_URL}/basicspacedata/query/class/gp_history/NORAD_CAT_ID/25544/EPOCH/>now-30/orderby/EPOCH desc/format/json',
    'satellite_catalog_recent': f'{BASE_URL}/basicspacedata/query/class/satcat/LAUNCH/>now-30/CURRENT/Y/orderby/LAUNCH desc/format/json',
    'recent_conjunctions': f'{BASE_URL}/basicspacedata/query/class/cdm_public/CREATION_DATE/>now-30/orderby/CREATION_DATE desc/format/json',
    'predicted_decay': f'{BASE_URL}/basicspacedata/query/class/decay/PRECEDENCE/4/orderby/DECAY_EPOCH desc/format/json'
}

def login(session):
    response = session.post(LOGIN_URL, data={'identity': USERNAME, 'password': PASSWORD})
    if response.status_code == 200:
        print("Successfully logged in.")
    else:
        raise Exception("Login failed!")

def fetch_and_save(session, endpoint, filename):
    response = session.get(endpoint)
    if response.status_code == 200:
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            df.to_csv(filename, index=False)
        else:
            print(f" No data returned for: {filename}")
    else:
        print("Failed request ({response.status_code}) for: {filename}")

if __name__ == "__main__":
    with requests.Session() as session:
        login(session)

        fetch_and_save(session, API_ENDPOINTS['latest_orbits'], "Dataset/latest_orbits.csv")
        fetch_and_save(session, API_ENDPOINTS['recent_history_iss'], "Dataset/iss_recent_history.csv")
        fetch_and_save(session, API_ENDPOINTS['satellite_catalog_recent'], "Dataset/satellite_catalog_recent.csv")
        fetch_and_save(session, API_ENDPOINTS['recent_conjunctions'], "Dataset/recent_conjunctions.csv")
        fetch_and_save(session, API_ENDPOINTS['predicted_decay'], "Dataset/predicted_decay.csv")
